In [21]:
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
from geopy.geocoders import Nominatim

## Geotagging the scraped Data

In [22]:
df = pd.read_csv('all_reviews.csv')
locations_df = pd.read_csv('locations_geotagged.csv')
beer_df = pd.read_csv('beer_deets.csv')

In [23]:
df

,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit
...,...,...,...,...,...,...
28141,Heineken,4.2,+2.1,Heineken Nederland,"Cerveja levemente amarga, mas suave. Não beber...",leonardocouto
28142,Taurus Pear Cider,5.0,+2.4,Aldi (UK),"Taurus, Taurus, Taurus that is my life now, I ...",PilsnerGuide144
28143,Taurus Dark Fruit,5.0,+2.5,Aldi (UK),I am a consistent Taurus drinker ! I have Taur...,PilsnerGuide144
28144,Great Lakes Ohio City Oatmeal Stout,1.7,-1.8,Great Lakes Brewing (Ohio),"This just tastes funny. Sort of a watery, kind...",EvanFriend


In [24]:
locations_df

,locations,latitude,longitude,country
0,"Bungay, Suffolk",52.456225,1.437341,United Kingdom
1,"Faversham, Kent",51.314409,0.891189,United Kingdom
2,"Bristol, Gloucestershire",51.852767,-2.255369,United Kingdom
3,"Round Rock, Texas",30.508235,-97.678893,United States of America
4,"Brooklyn, New York",40.650104,-73.949582,United States of America
...,...,...,...,...
2346,"Rockville, Virginia",39.621471,-79.704774,United States of America
2347,"Axberg, Austria",46.953826,15.760800,Austria
2348,"Royal Oak, Michigan",42.489480,-83.144649,United States of America
2349,"Bécancour, Quebec",46.339343,-72.433205,Canada


In [25]:
geolocator = Nominatim(user_agent="None")

In [26]:
locations_new = pd.DataFrame(df.brewery_location[~df.brewery_location.isin(locations_df.locations)].unique(),columns=['locations'])
print(len(locations_new), "new locations to search for! Will take", len(locations_new)/2, "seconds to run.")

21 new locations to search for! Will take 10.5 seconds to run.


In [27]:
def tryconvert(x):
        try:
            return geolocator.geocode(x).latitude
        except:
            return None
    

In [28]:
def tryconvertlong(x):
        try:
            return geolocator.geocode(x).longitude
        except:
            return None

In [29]:
locations_new["latitude"] = locations_new.locations.apply(lambda x: tryconvert(x))

In [30]:
locations_new["longitude"] = locations_new.locations.apply(lambda x: tryconvertlong(x))

In [31]:
countries = []
for row in locations_new.index:
    try:
        loc = geolocator.reverse([locations_new.iloc[row,1],locations_new.iloc[row,2]], language='en')
        countries.append(loc.raw['address']['country'])
    except:
        countries.append(None)
        
    

In [32]:
locations_new['country'] = countries

In [33]:
locations_new

,locations,latitude,longitude,country
0,"Untersiemau-Birkach, Bavaria",50.165451,11.007970,Germany
1,"LaFayette, New York",42.892289,-76.105202,United States of America
2,"Ogden, Utah",41.223005,-111.973843,United States of America
3,"Clermont, New Jersey",39.990668,-74.799607,United States of America
4,"Sechelt, British Columbia",49.472115,-123.763124,Canada
5,"Little Current, Ontario",45.980049,-81.926799,Canada
6,"New City, New York",41.149983,-73.989898,United States of America
7,"Pennigsehl, Lower Saxony",52.644061,9.027508,Germany
8,"Excelsior, Minnesota",44.903296,-93.566345,United States of America
9,"Lübbecke, North Rhine-Westphalia",52.302721,8.618305,Germany


In [34]:
locations_df = pd.concat([locations_df,locations_new])
locations_df.to_csv('locations_geotagged.csv',index=False)

## Merging and cleaning

In [35]:
beer_df

,return_string,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number,name_found
0,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,🇬🇧,Bitter - Ordinary / Best,4.3,49,95,3.25,415.0,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,🇬🇧,Bitter - Ordinary / Best,4.5,34,59,3.05,1083.0,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,🇬🇧,IPA,6.0,95,97,3.74,59.0,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,🇬🇧,IPA - English,6.1,46,47,3.22,420.0,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,🇬🇧,Strong Ale - English,7.0,45,46,3.18,409.0,Shepherd Neame Christmas Ale
...,...,...,...,...,...,...,...,...,...
15129,Cycle Snickerdoodle Dandy DOS\n🇺🇸 Stout - Impe...,🇺🇸,Stout - Imperial Flavored,NaN,-,-,4.00,1.0,Cycle Snickerdoodle Dandy DOS
15130,Block 15 Wellspring\n🇺🇸 Belgian Ale - Pale / G...,🇺🇸,Belgian Ale - Pale / Golden,NaN,-,-,3.60,1.0,Block 15 Wellspring
15131,Tree House Curiosity 96 (Ninety-Six)\n🇺🇸 IIPA ...,🇺🇸,IIPA DIPA - Imperial / Double IPA,NaN,-,-,4.00,1.0,Tree House Curiosity 96 (Ninety-Six)
15132,Sommerbecker Dachs Pils\n🇩🇪 Pilsener / Pils / ...,🇩🇪,Pilsener / Pils / Pilsner,0.0,17,41,2.92,14.0,Sommerbecker Dachs Pils


In [36]:
# Creating DF with all details
df_all = pd.merge(df,locations_df,how='left',left_on='brewery_location',right_on='locations')
df_all = pd.merge(df_all, beer_df, how='left', left_on='beer_name',right_on='name_found')
df_all


,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name,locations,latitude,longitude,country,return_string,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number,name_found
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit,"Bungay, Suffolk",52.456225,1.437341,United Kingdom,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,🇬🇧,Bitter - Ordinary / Best,4.3,49,95,3.25,415.0,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,🇬🇧,Bitter - Ordinary / Best,4.5,34,59,3.05,1083.0,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG,"Bristol, Gloucestershire",51.852767,-2.255369,United Kingdom,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,🇬🇧,IPA,6.0,95,97,3.74,59.0,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,🇬🇧,IPA - English,6.1,46,47,3.22,420.0,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,🇬🇧,Strong Ale - English,7.0,45,46,3.18,409.0,Shepherd Neame Christmas Ale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28414,Heineken,4.2,+2.1,Heineken Nederland,"Cerveja levemente amarga, mas suave. Não beber...",leonardocouto,+2.1,33.814488,130.890462,Japan,Heineken\n🇳🇱 Pale Lager - Intl / Premium • 5.0...,🇳🇱,Pale Lager - Intl / Premium,5.0,3,10,2.36,3756.0,Heineken
28415,Taurus Pear Cider,5.0,+2.4,Aldi (UK),"Taurus, Taurus, Taurus that is my life now, I ...",PilsnerGuide144,+2.4,NaN,9.991238,NaN,Taurus Pear Cider\n🇬🇧 Perry • 5.3%\n-\n-\n3.19...,🇬🇧,Perry,5.3,-,-,3.19,8.0,Taurus Pear Cider
28416,Taurus Dark Fruit,5.0,+2.5,Aldi (UK),I am a consistent Taurus drinker ! I have Taur...,PilsnerGuide144,+2.5,50.814444,14.361805,Czech Republic,Taurus Dark Fruit\n🇬🇧 Apple Cider - Flavored •...,🇬🇧,Apple Cider - Flavored,4.0,-,-,3.64,5.0,Taurus Dark Fruit
28417,Great Lakes Ohio City Oatmeal Stout,1.7,-1.8,Great Lakes Brewing (Ohio),"This just tastes funny. Sort of a watery, kind...",EvanFriend,-1.8,48.780981,10.037026,Germany,Great Lakes Ohio City Oatmeal Stout\n🇺🇸 Stout ...,🇺🇸,Stout - Oatmeal,5.4,84,69,3.49,184.0,Great Lakes Ohio City Oatmeal Stout


In [37]:
# drop excess columns
df_all.drop(["locations","return_string","name_found"],axis=1,inplace=True)
# drop rows with ratings instead of location
df_all = df_all[~df_all['brewery_location'].str.len()<4]
# drop all NaNs
print(len(df_all) - len(df_all.dropna()), "columns with NaNs dropped.")
df_all.dropna(inplace=True)


2169 columns with NaNs dropped.


In [38]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26250 entries, 0 to 28418
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   beer_name         26250 non-null  object 
 1   beer_rating       26250 non-null  float64
 2   brewery_location  26250 non-null  object 
 3   brewery_name      26250 non-null  object 
 4   review_content    26250 non-null  object 
 5   reviewer_name     26250 non-null  object 
 6   latitude          26250 non-null  float64
 7   longitude         26250 non-null  float64
 8   country           26250 non-null  object 
 9   flag              26250 non-null  object 
 10  type              26250 non-null  object 
 11  abv               26250 non-null  float64
 12  rb_overall_score  26250 non-null  object 
 13  rb_style_score    26250 non-null  object 
 14  rb_user_rating    26250 non-null  object 
 15  rb_rating_number  26250 non-null  float64
dtypes: float64(5), object(11)
memory usage: 

## Deal with Non-English Descriptions

## Save down cleaned data

In [39]:
df_all.to_csv("data_clean.csv",index=False)